# **Modelado Avanzado y Ajuste de Hiperparámetros**

Este notebook es la extensión avanzada del trabajo iniciado en los modelos baseline (`andrea-model-training-baseline.ipynb`). Mientras que allí usábamos modelos clásicos (LogisticRegression, MultinomialNB, etc.) con vectorización textual (TF-IDF, CountVectorizer), aquí damos el salto a:

- Técnicas modernas de NLP: embeddings + LSTM
- Ajuste de hiperparámetros implícito mediante regularización
- Evaluación formal del overfitting, requisito específico del proyecto

**Objetivo**: Crear un modelo más expresivo (deep learning) y evaluar su generalización, asegurando que no haya overfitting significativo (diferencia ≤ 5pp en F1 entre entrenamiento y test).

## **Librerías y Carga de datos**

#### **¿Qué datos usamos y por qué?**
- Usamos el dataset `text_cleaned`, que proviene del pipeline de preprocesamiento.
- La columna `text_cleaned` fue limpiada: sin emojis, contracciones, URLs, ruido.
- Elegimos `IsToxic` como etiqueta, siguiendo la misma lógica del baseline, para poder comparar.

🧠 Esto garantiza continuidad metodológica: estamos evaluando si el modelo avanzado realmente mejora el baseline en condiciones comparables.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, json, os, re
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
# Parámetros
max_words = 10000
max_len = 150
embedding_dim = 100

In [ ]:
# 📥 Cargar datos
DATA_DIR = '../processed_data/text_cleaned'
X_train = pd.read_csv(f'{DATA_DIR}/X_train.csv').squeeze()
X_test = pd.read_csv(f'{DATA_DIR}/X_test.csv').squeeze()
y_train = pd.read_csv(f'{DATA_DIR}/y_train.csv')['IsToxic']
y_test = pd.read_csv(f'{DATA_DIR}/y_test.csv')['IsToxic']

## **Tokenización y secuenciación**

### **¿Qué hacemos aquí?**

- **Tokenizer**: construye un vocabulario a partir del texto. Cada palabra se convierte en un entero.
- **OOV token**: maneja palabras desconocidas en test con un token especial.

- **pad_sequences**: asegura que todas las secuencias tengan la misma longitud (truncando o completando con ceros al final).

### **¿Por qué es necesario?**

🧠 Los modelos de deep learning no entienden strings. Necesitan secuencias de enteros, de longitud fija. Este proceso traduce lenguaje natural a formato numérico estructurado.

In [ ]:
# 🔤 Tokenización y secuenciación
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

## **Definición del modelo avanzado (Embedding + LSTM)**

### **Arquitectura detallada**

| Capa                 | Función                                                              |
| -------------------- | -------------------------------------------------------------------- |
| `Embedding`          | Transforma cada palabra (índice) en un vector denso y aprendible     |
| `Bidirectional LSTM` | Modelo secuencial que lee el texto hacia adelante y hacia atrás      |
| `Dense + ReLU`       | Capa oculta completamente conectada                                  |
| `Dropout`            | Previene overfitting desconectando neuronas al azar en entrenamiento |
| `Dense + sigmoid`    | Salida binaria (probabilidad de toxicidad)                           |


### **Técnicas avanzadas usadas aquí:**

- Embeddings aprendibles: los vectores de palabra se ajustan durante el entrenamiento.

- Regularización L2 (kernel_regularizer=l2(0.01)): penaliza pesos grandes → evita que el modelo se sobreajuste.

- Dropout: mejora la generalización.

- Bidirectional: LSTM lee el texto en ambas direcciones, capturando más contexto semántico.

🧠 Este modelo es más expresivo y flexible que los modelos lineales del baseline, por lo que se espera mejor rendimiento, pero también mayor riesgo de overfitting si no se controla bien.

In [ ]:
# 🧠 Modelo LSTM con embeddings
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(0.01))),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## **Entrenamiento con early stopping**

- Entrenamos con el 80% del `train`, validamos en el 20% restante.
- Si la `val_loss` no mejora durante 3 épocas, se detiene el entrenamiento.
- Se restauran los mejores pesos (no los últimos) para evitar sobreajuste final.


🧠 Early stopping es una técnica de regularización práctica que evita el sobreentrenamiento.

In [ ]:
# ⏳ Entrenamiento con early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=32,
    callbacks=[early_stop],
    verbose=2
)

## **Evaluación y detección de overfitting**

###**¿Qué hacemos?**
- Generamos predicciones binarias (prob > 0.5).
- Calculamos F1-score, la métrica más relevante para clasificación desequilibrada.
- Medimos la diferencia absoluta entre el F1 de entrenamiento y test.

###**¿Por qué 5pp como umbral?**
Un delta mayor a 5 puntos porcentuales en F1 (por ejemplo, 0.85 vs 0.75) indica que el modelo aprendió demasiado bien el train, pero no logra generalizar → overfitting.

In [ ]:
# 📈 Evaluación
train_preds = (model.predict(X_train_pad) > 0.5).astype(int)
test_preds = (model.predict(X_test_pad) > 0.5).astype(int)

train_f1 = f1_score(y_train, train_preds)
test_f1 = f1_score(y_test, test_preds)

diff = abs(train_f1 - test_f1) * 100
print(f"Train F1: {train_f1:.4f}  |  Test F1: {test_f1:.4f}  | Δ = {diff:.2f}pp")
if diff <= 5:
    print("✅ No hay overfitting significativo")
else:
    print("❌ Posible overfitting: diferencia superior a 5pp")


## **Reporte final y curvas**

Incluye:
- Precision: qué proporción de los positivos predichos eran realmente positivos
- Recall: qué proporción de los positivos reales fueron detectados
- F1-score: media armónica entre ambas
- Soporte: cuántos ejemplos hay por clase

Dos gráficos clave:

- Accuracy Train vs Validation
  - Si divergen mucho → el modelo está memorizando

- Loss Train vs Validation
  - Un val_loss creciente mientras train_loss baja → clara señal de overfitting

🧠 Estas gráficas son herramientas esenciales para el diagnóstico temprano de problemas de generalización.

In [ ]:
# 🧾 Reporte detallado
print("\nClassification Report (Test):")
print(classification_report(y_test, test_preds))

In [ ]:

# 📊 Visualización de curvas de entrenamiento
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.tight_layout()
plt.show()